In [31]:
from math import *
import numpy as np
import matplotlib.pyplot as plt
import cv2
from sklearn import linear_model, decomposition, datasets
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from matplotlib.ticker import NullFormatter

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from keras.datasets import imdb
from keras.utils import np_utils
from keras.optimizers import RMSprop

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 10.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

In [2]:
yone = np.load('./onedollardata/label.npy')
trajonet = np.load('./onedollardata/trajdata.npy')
ximgt = np.load('./onedollardata/imgdata.npy')

In [3]:

print yone.shape
print trajonet[0].shape
print ximgt.shape

(5280,)
(32, 2)
(5280, 1, 28, 28)


In [9]:
maxlen = 0
xtraintemp = []
for i in range(len(trajonet)):
    if maxlen<trajonet[i].shape[0]:
        maxlen = trajonet[i].shape[0]
print maxlen
for i in range(len(trajonet)):
    if trajonet[i].shape[0]<209:
        tt = np.concatenate((trajonet[i],np.zeros((209-trajonet[i].shape[0],2))),axis = 0)
    xtraintemp.append(tt)
Xone = np.array(xtraintemp).reshape((-1,maxlen,2))
print Xone.shape

209
(5280, 209, 2)


### split dataset into train validation and test

In [22]:
np.random.seed(100)
maskTrain = np.random.choice(5280,2640,replace=False)
afterTrain = np.setdiff1d(range(5280),maskTrain)
maskValidation = np.random.choice(afterTrain,1320,replace=False)
maskTest = np.setdiff1d(afterTrain,maskValidation)
print maskTrain

[1639 4204 3264 ..., 2159 1609  763]


In [28]:
numClass = 16
Xtrain = Xone[maskTrain,:,:]
Xvalid = Xone[maskValidation,:,:]
Xtest = Xone[maskTest,:,:]
Ytrain = np_utils.to_categorical(yone[maskTrain],numClass)
Yvalid = np_utils.to_categorical(yone[maskValidation],numClass)
Ytest = np_utils.to_categorical(yone[maskTest],numClass)
print Xtrain.shape, Xvalid.shape, Xtest.shape
print Ytrain.shape, Yvalid.shape, Ytest.shape

(2640, 209, 2) (1320, 209, 2) (1320, 209, 2)
(2640, 16) (1320, 16) (1320, 16)


In [34]:
model = Sequential()
model.add(LSTM(132, input_shape=(maxlen, 2)))
model.add(Dense(numClass,activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer = RMSprop(),metrics=['accuracy'])
print "training.."
model.fit(Xtrain,Ytrain,batch_size = 20,nb_epoch = 20, validation_data=[Xvalid,Yvalid])

training..
Train on 2640 samples, validate on 1320 samples
Epoch 1/20
2640/2640 [==============================] - 42s - loss: 2.7746 - acc: 0.0648 - val_loss: 2.7240 - val_acc: 0.0909
Epoch 2/20
2640/2640 [==============================] - 44s - loss: 2.5866 - acc: 0.1144 - val_loss: 2.3102 - val_acc: 0.1621
Epoch 3/20
2640/2640 [==============================] - 43s - loss: 2.2117 - acc: 0.2019 - val_loss: 2.5704 - val_acc: 0.1697
Epoch 4/20
2640/2640 [==============================] - 43s - loss: 2.2497 - acc: 0.2208 - val_loss: 2.0719 - val_acc: 0.2242
Epoch 5/20
2640/2640 [==============================] - 43s - loss: 2.0007 - acc: 0.2564 - val_loss: 1.8092 - val_acc: 0.2674
Epoch 6/20
2640/2640 [==============================] - 46s - loss: 1.8397 - acc: 0.2777 - val_loss: 2.0166 - val_acc: 0.2788
Epoch 7/20
2640/2640 [==============================] - 43s - loss: 2.3114 - acc: 0.2189 - val_loss: 1.9121 - val_acc: 0.2432
Epoch 8/20
2640/2640 [==============================] - 42s